# New Section


In [4]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, models, transforms
from torch.utils.data import DataLoader
import time
import copy

# Check if GPU is available
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")
######################################################################Reyyan

Using device: cuda:0


In [8]:
import os

def rename_files(directory):
    for root, dirs, files in os.walk(directory):
        for file in files:
            if ' ' in file:
                new_file = file.replace(' ', '_')
                os.rename(os.path.join(root, file), os.path.join(root, new_file))

# Define data directory
data_dir = './data'  # Update this to your local data directory

# Rename files in train and val directories
rename_files(os.path.join(data_dir, 'train'))
rename_files(os.path.join(data_dir, 'val'))

In [12]:
import os
import shutil
from sklearn.model_selection import train_test_split

def split_data(data_dir, train_dir='train', val_dir='val', val_size=0.2):
    train_path = os.path.join(data_dir, train_dir)
    val_path = os.path.join(data_dir, val_dir)
    
    # Create val directory if it doesn't exist
    if not os.path.exists(val_path):
        os.makedirs(val_path)
    
    for class_name in os.listdir(train_path):
        class_train_path = os.path.join(train_path, class_name)
        class_val_path = os.path.join(val_path, class_name)
        
        # Create class directory in val if it doesn't exist
        if not os.path.exists(class_val_path):
            os.makedirs(class_val_path)
        
        # Get list of all files in the class directory
        files = os.listdir(class_train_path)
        train_files, val_files = train_test_split(files, test_size=val_size, random_state=42)
        
        # Move validation files to val directory
        for file in val_files:
            shutil.move(os.path.join(class_train_path, file), os.path.join(class_val_path, file))

# Define data directory
data_dir = './data'  # Update this to your local data directory

# Split data
split_data(data_dir)

In [13]:
import os
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

# Define data directory
data_dir = './data'  # Update this to your local data directory

# Print directory contents for debugging
for root, dirs, files in os.walk(data_dir):
    print(root, dirs, files)

# Define data transformations
data_transforms = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

# Load datasets
image_datasets = {x: datasets.ImageFolder(root=os.path.join(data_dir, x), transform=data_transforms[x])
                  for x in ['train', 'val']}

# Create data loaders
dataloaders = {x: DataLoader(image_datasets[x], batch_size=32, shuffle=True, num_workers=4)
               for x in ['train', 'val']}

# Dataset sizes
dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'val']}

# Class names
class_names = image_datasets['train'].classes
print(f"Classes: {class_names}")


./data ['train', 'val'] []
./data/train ['class1', 'class2'] []
./data/train/class1 [] ['587_(2).png', '020_(3).png', '548_(2).png', '235_(2).png', '446_(3).png', '1601.png', '047_(3).png', '254_(2).png', '357_(3).png', '574_(2).png', '036_(4).png', '013_(3).png', '589_(2).png', '072_(3).png', '025_(3).png', '008_(3).png', '022_(4).png', '755_(2).png', '031_(4).png', '1194_(2).png', '023_(5).png', '223_(2).png', '290_(2).png', '045_(3).png', '452_(2).png', '092_(2).png', '504_(2).png', '681_(2).png', '697_(2).png', '177_(2).png', '355_(3).png', '671_(2).png', '096_(2).png', '019_(4).png', '004_(5).png', '541_(2).png', '202_(2).png', '576_(2).png', '611_(2).png', '089_(2).png', '110_(3).png', '747_(2).png', '429_(3).png', '082_(3).png', '562_(2).png', '266_(2).png', '241_(2).png', '442_(3).png', '386_(3).png', '093_(3).png', '247_(2).png', '128_(2).png', '171_(2).png', '273_(2).png', '331_(2).png', '352_(2).png', '538_(2).png', '142_(2).png', '162_(3).png', '231_(2).png', '667_(2).png',

In [14]:
# Load pre-trained ResNet18 model
model = models.resnet18(pretrained=True)

# Freeze all layers
for param in model.parameters():
    param.requires_grad = False

# Modify the final layer
num_features = model.fc.in_features
model.fc = nn.Linear(num_features, len(class_names))

# Move model to the appropriate device
model = model.to(device)


/home/himan/miniconda3/envs/newenv/lib/python3.11/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/himan/miniconda3/envs/newenv/lib/python3.11/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /home/himan/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:00<00:00, 67.3MB/s]


In [15]:
# Define loss function
criterion = nn.CrossEntropyLoss()

# Define optimizer (only for the final layer)
optimizer = optim.Adam(model.fc.parameters(), lr=0.001)

# Learning rate scheduler
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)


In [16]:
def train_model(model, criterion, optimizer, scheduler, num_epochs=25):
    since = time.time()

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        print(f'Epoch {epoch}/{num_epochs - 1}')
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data
            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                # Zero the parameter gradients
                optimizer.zero_grad()

                # Forward
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    # Backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # Statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

            if phase == 'train':
                scheduler.step()

            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]

            print(f'{phase} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')

            # Deep copy the model
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())

        print()

    time_elapsed = time.time() - since
    print(f'Training complete in {time_elapsed // 60:.0f}m {time_elapsed % 60:.0f}s')
    print(f'Best val Acc: {best_acc:.4f}')

    # Load best model weights
    model.load_state_dict(best_model_wts)
    return model


In [17]:
# Train the model
num_epochs = 25
model = train_model(model, criterion, optimizer, scheduler, num_epochs=num_epochs)


Epoch 0/24
----------
train Loss: 0.3253 Acc: 0.8800
val Loss: 0.0790 Acc: 0.9887

Epoch 1/24
----------
train Loss: 0.1115 Acc: 0.9744
val Loss: 0.0442 Acc: 0.9972

Epoch 2/24
----------
train Loss: 0.0859 Acc: 0.9787
val Loss: 0.0288 Acc: 1.0000

Epoch 3/24
----------
train Loss: 0.0713 Acc: 0.9780
val Loss: 0.0208 Acc: 1.0000

Epoch 4/24
----------
train Loss: 0.0592 Acc: 0.9822
val Loss: 0.0177 Acc: 1.0000

Epoch 5/24
----------
train Loss: 0.0521 Acc: 0.9879
val Loss: 0.0136 Acc: 1.0000

Epoch 6/24
----------
train Loss: 0.0508 Acc: 0.9865
val Loss: 0.0125 Acc: 1.0000

Epoch 7/24
----------
train Loss: 0.0515 Acc: 0.9830
val Loss: 0.0116 Acc: 1.0000

Epoch 8/24
----------
train Loss: 0.0434 Acc: 0.9872
val Loss: 0.0128 Acc: 1.0000

Epoch 9/24
----------
train Loss: 0.0507 Acc: 0.9851
val Loss: 0.0112 Acc: 1.0000

Epoch 10/24
----------
train Loss: 0.0389 Acc: 0.9908
val Loss: 0.0116 Acc: 1.0000

Epoch 11/24
----------
train Loss: 0.0405 Acc: 0.9858
val Loss: 0.0111 Acc: 1.0000

Ep

In [18]:
# # Save the trained model
# model_save_path = '/content/drive/MyDrive/path_to_save_directory/model_weights.pth'
# torch.save(model.state_dict(), model_save_path)
# print(f'Model saved to {model_save_path}')

model_save_dir = './saved_models'  # Update this to your desired local directory
os.makedirs(model_save_dir, exist_ok=True)
model_save_path = os.path.join(model_save_dir, 'model_weights.pth')

# Save the trained model
torch.save(model.state_dict(), model_save_path)
print(f'Model saved to {model_save_path}')


Model saved to ./saved_models/model_weights.pth


In [ ]:
# Load the model for inference
model = models.resnet18(pretrained=False)
num_features = model.fc.in_features
model.fc = nn.Linear(num_features, len(class_names))
model.load_state_dict(torch.load(model_save_path))
model = model.to(device)
model.eval()

::contentReference[oaicite:0]{index=0}

